# Introduccion

El servicio de venta de autos usados Rusty Bargain está desarrollando una aplicación para atraer nuevos clientes. Gracias a esa app, puedes averiguar rápidamente el valor de mercado de tu coche. Se tiene acceso al historial: especificaciones técnicas, versiones de equipamiento y precios. Se creará un modelo que determine el valor de mercado.
A Rusty Bargain le interesa:
- la calidad de la predicción;
- la velocidad de la predicción;
- el tiempo requerido para el entrenamiento

## Preparación de datos

In [98]:
# importacion de librerias
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
import numpy as np



### Revision general de datos

In [99]:
#Carga de Datos

df = pd.read_csv('/datasets/car_data.csv')
df.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,24/03/2016 11:52,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,24/03/2016 00:00,0,70435,07/04/2016 03:16
1,24/03/2016 10:58,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,24/03/2016 00:00,0,66954,07/04/2016 01:46
2,14/03/2016 12:52,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,14/03/2016 00:00,0,90480,05/04/2016 12:47
3,17/03/2016 16:54,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,17/03/2016 00:00,0,91074,17/03/2016 17:40
4,31/03/2016 17:25,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,31/03/2016 00:00,0,60437,06/04/2016 10:17


In [100]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

 - Los datos presentan valores ausentes que deberam ser tratados previo al analisis. 
 - El tipo de datos es correcto para cada caracteristica

In [101]:
# Analisis de suplicados
df.duplicated().sum()

262

In [102]:
# ELiminacion de duplicados

df = df.drop_duplicates(ignore_index = True)
df.shape

(354107, 16)

In [103]:
#Se eliminan las caracteristicas que no aportan informacion al analisis (no estan relacionadas con el precio):('DateCrawled', 'DateCreated', 'NumberOfPictures', 'PostalCode', 'LastSeen')

df = df.drop(['DateCrawled', 'DateCreated', 'NumberOfPictures', 'PostalCode', 'LastSeen'], axis=1)

        

<div class="alert alert-block alert-success">
    <b>Buen trabajo! </div>

In [104]:
#se cambian los nombres de las columnas a minusculas

df.columns = df.columns.str.lower()


### Tratamiento de valores ausentes


In [105]:
#Dado que la mayoria de los datos con valores ausentes estan relacionados con el modelo del vehiculo, se utilizara este ultimo para predecir los valores ausentes. 
#Dado que el tipo de modelo no se puede inferir directamente de los datos disponibles, se eliminan dichos ausentes y luego se utilizan los datos resultantes para predecir 

df['model'].isna()

df = df.dropna(subset=['model'])

df.shape

(334406, 11)

In [106]:
# Se rellenan valores ausentes de "fueltype" a partir del modelo y marca del vehiculo

In [107]:


df_sin_fuel_nan =  df.dropna(subset=['fueltype']) # datos sin vacios en "fueltype"
dicc_fuel = dict(zip(zip(df_sin_fuel_nan['model'], df_sin_fuel_nan['brand']), df_sin_fuel_nan['fueltype']))#diccionario que mapea modelo y marca con fuel




In [108]:
df['fueltype'] = df['fueltype'] .fillna(df.apply(lambda x: dicc_fuel.get((x['model'], x['brand']), x['fueltype']), axis=1))





In [109]:
df['fueltype'].isna().sum()

0

 - se eliminarion los ausentes de fuel type

In [110]:
# Se rellenan valores ausentes de "tipo de vehiculo" a partir del modelo y marca del vehiculo

In [111]:

df_sin_tipo_nan =  df.dropna(subset=['vehicletype']) # datos sin vacios en "tipo de vehiculo"
dicc_tipo = dict(zip(zip(df_sin_fuel_nan['model'], df_sin_fuel_nan['brand']), df_sin_fuel_nan['vehicletype']))#diccionario que mapea modelo y marca con fuel

df['vehicletype'] = df['vehicletype'].fillna(df.apply(lambda x: dicc_tipo.get((x['model'], x['brand']), x['vehicletype']), axis=1))
df['vehicletype'].isna().sum()


1251

- Aun quedan valores NAN en tipo de vehiculos pero se redujo a solo 1251 registros

In [112]:
# Se rellenan valores ausentes de "gearbox" a partir del modelo y marca del vehiculo

In [113]:

df_sin_box_nan =  df.dropna(subset=['gearbox']) # datos sin vacios en "tipo de caja"
dicc_box = dict(zip(zip(df_sin_fuel_nan['model'], df_sin_fuel_nan['brand']), df_sin_fuel_nan['gearbox']))#diccionario que mapea modelo y marca con fuel

df['gearbox'] = df['gearbox'].fillna(df.apply(lambda x: dicc_box.get((x['model'], x['brand']), x['gearbox']), axis=1))
df['gearbox'].isna().sum()



269

- Aun quedan algunos valores ausentes en tipo de transmision pero se redujeron a tan solo 269 registros

In [114]:
#se rellenan valores ausentes de reparaciones
df

,price,vehicletype,registrationyear,gearbox,power,model,mileage,registrationmonth,fueltype,brand,notrepaired
0,480,wagon,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN
2,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN
3,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no
4,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no
5,650,sedan,1995,manual,102,3er,150000,10,petrol,bmw,yes
...,...,...,...,...,...,...,...,...,...,...,...
354101,1150,bus,2000,manual,0,zafira,150000,3,petrol,opel,no
354102,0,NaN,2005,manual,0,colt,150000,7,petrol,mitsubishi,yes
354104,1199,convertible,2000,auto,101,fortwo,125000,3,petrol,smart,no
354105,9200,bus,1996,manual,102,transporter,150000,3,gasoline,volkswagen,no


In [115]:
#se crea una tabla con los kilometrajes y el valor de reparacion que aparece con mayor frecuncia
tabla = df.groupby('mileage')['notrepaired'].apply(lambda x: x.mode()[0]).reset_index()
tabla

,mileage,notrepaired
0,5000,no
1,10000,no
2,20000,no
3,30000,no
4,40000,no
5,50000,no
6,60000,no
7,70000,no
8,80000,no
9,90000,no


In [116]:
# Unir el DataFrame original con la tabla de moda utilizando el método merge()
df = pd.merge(df, tabla, on='mileage', how='left', suffixes=('_original', '_moda'))
df['notrepaired_moda'].unique()


array(['no'], dtype=object)

In [117]:
df['notrepaired']=df['notrepaired_original'].fillna(df['notrepaired_moda'])

#dfbt['notrepaired_original'].unique()
# df['notrepaired']=df['notrepaired'].fillna(dfbt['notrepaired_original'])


In [118]:

df['notrepaired'].value_counts()

no     300715
yes     33691
Name: notrepaired, dtype: int64

In [119]:
df

,price,vehicletype,registrationyear,gearbox,power,model,mileage,registrationmonth,fueltype,brand,notrepaired_original,notrepaired_moda,notrepaired
0,480,wagon,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,no,no
1,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,no,no
2,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,no,no
3,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,no,no
4,650,sedan,1995,manual,102,3er,150000,10,petrol,bmw,yes,no,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...
334401,1150,bus,2000,manual,0,zafira,150000,3,petrol,opel,no,no,no
334402,0,NaN,2005,manual,0,colt,150000,7,petrol,mitsubishi,yes,no,yes
334403,1199,convertible,2000,auto,101,fortwo,125000,3,petrol,smart,no,no,no
334404,9200,bus,1996,manual,102,transporter,150000,3,gasoline,volkswagen,no,no,no


In [120]:
df.columns

Index(['price', 'vehicletype', 'registrationyear', 'gearbox', 'power', 'model',
       'mileage', 'registrationmonth', 'fueltype', 'brand',
       'notrepaired_original', 'notrepaired_moda', 'notrepaired'],
      dtype='object')

In [121]:
df = df.drop(['notrepaired_moda', 'notrepaired_original'], axis=1)


In [122]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 334406 entries, 0 to 334405
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   price              334406 non-null  int64 
 1   vehicletype        333155 non-null  object
 2   registrationyear   334406 non-null  int64 
 3   gearbox            334137 non-null  object
 4   power              334406 non-null  int64 
 5   model              334406 non-null  object
 6   mileage            334406 non-null  int64 
 7   registrationmonth  334406 non-null  int64 
 8   fueltype           334406 non-null  object
 9   brand              334406 non-null  object
 10  notrepaired        334406 non-null  object
dtypes: int64(5), object(6)
memory usage: 30.6+ MB


In [123]:
# LUEGO DEHABER COMPLETADO la mayoria de los datos ausentes eliminamos los pocos que no pudieron completarse
df = df.dropna(subset=['vehicletype','gearbox'])

In [124]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 332906 entries, 0 to 334405
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   price              332906 non-null  int64 
 1   vehicletype        332906 non-null  object
 2   registrationyear   332906 non-null  int64 
 3   gearbox            332906 non-null  object
 4   power              332906 non-null  int64 
 5   model              332906 non-null  object
 6   mileage            332906 non-null  int64 
 7   registrationmonth  332906 non-null  int64 
 8   fueltype           332906 non-null  object
 9   brand              332906 non-null  object
 10  notrepaired        332906 non-null  object
dtypes: int64(5), object(6)
memory usage: 30.5+ MB


### Analisis de datos contenidos

In [125]:
# Se analiza dato tipo de vehiculo
df['gearbox'].value_counts()

manual    267789
auto       65117
Name: gearbox, dtype: int64

In [126]:
# Se analizan los datos de la potencia
df['power'].value_counts()


0       33186
75      22710
60      14982
150     13937
101     12754
        ...  
540         1
416         1
1288        1
6045        1
551         1
Name: power, Length: 683, dtype: int64

In [127]:
# Se eliminan los valores de potencia mayores a 2000 y menores a 10 ya que carecen de sentido

df = df[(df['power']>= 2000) | (df['power']<= 10)]


In [128]:
# Se analizan los datos de modelos


In [129]:
df['model'].unique()

array(['golf', 'ka', 'kangoo', 'corsa', 'signum', 'astra', 'polo', 'a4',
       'insignia', 'combo', '3er', '100', 'other', 'v40', 'a3', '7er',
       'zafira', 'e_klasse', 'c_klasse', 'espace', 'sharan', 'a_klasse',
       'seicento', 'transporter', 'c1', 'fox', '6_reihe', 'c_max',
       '2_reihe', '5er', 'cooper', '500', 'lupo', 'x_reihe', 'punto',
       'mondeo', 'a2', 'twingo', 'fiesta', 'qashqai', 'fabia', 's_klasse',
       'focus', 'ibiza', 'civic', '1er', 'megane', 'caddy', 'kalos',
       'laguna', 'transit', 'omega', 'cayenne', 'b_klasse', 'micra',
       'corolla', 'clio', 'vito', 'touran', 'cr_reihe', 'clk', 'meriva',
       'fortwo', 'escort', 'cordoba', 'sprinter', 'passat', 'modus', 'a5',
       '3_reihe', 'v70', 'octavia', 'kaefer', 'agila', 'a6', 'antara',
       'arosa', 'touareg', 'primera', 'picanto', 'stilo', 'bora',
       'scenic', 'q7', 'berlingo', 'yaris', '1_reihe', 'leon', 'jetta',
       'almera', 'tigra', 'ptcruiser', 'samara', 'c3', 'galant', 'verso',
  

In [130]:
# SE anlaizan los datos contenidos en millaje


In [131]:
df['mileage'].unique()

array([150000,   5000, 100000,  10000,  90000,  60000, 125000,  80000,
        40000,  20000,  30000,  70000,  50000])

In [132]:
# Se analizan los datos del mes de registracion


In [133]:
df['registrationmonth'].unique()

array([ 0,  2,  1,  8,  3,  9, 12, 10,  4,  7,  6, 11,  5])

In [134]:
# 
df['registrationmonth'].value_counts(normalize = True)

0     0.337459
3     0.068661
6     0.065124
5     0.058681
4     0.058261
7     0.057062
11    0.056913
10    0.055924
9     0.051278
12    0.051158
8     0.050289
1     0.049150
2     0.040040
Name: registrationmonth, dtype: float64

- Dado que hay un alto porcentaje con valor de mes = 0 que carece de sentido y que el mes no tiene un alta correlacion con el precio, se eliminara esta caracateristica pra el analisis
    
   

In [ ]:

df = df.drop(['registrationmonth'],axis=1)

In [136]:
# Se analizan los datos de tipo de combustible
df['fueltype'].unique()

array(['petrol', 'gasoline', 'other', 'lpg', 'hybrid', 'cng', 'electric'],
      dtype=object)

In [137]:
#Se analizan los datos de marca
df['brand'].unique()

array(['volkswagen', 'ford', 'renault', 'opel', 'audi', 'bmw', 'fiat',
       'mitsubishi', 'volvo', 'mercedes_benz', 'nissan', 'chevrolet',
       'citroen', 'peugeot', 'mazda', 'mini', 'skoda', 'seat', 'porsche',
       'honda', 'daewoo', 'toyota', 'hyundai', 'lancia', 'smart',
       'jaguar', 'kia', 'chrysler', 'rover', 'lada', 'trabant', 'suzuki',
       'alfa_romeo', 'dacia', 'jeep', 'daihatsu', 'saab', 'subaru',
       'land_rover'], dtype=object)

In [138]:
df['notrepaired'].unique()

array(['no', 'yes'], dtype=object)

### Analisis de correlacion con el precio

In [139]:

df.corr()

,price,registrationyear,power,mileage
price,1.000000,0.019421,0.024823,-0.105795
registrationyear,0.019421,1.000000,0.003813,-0.079645
power,0.024823,0.003813,1.000000,-0.009342
mileage,-0.105795,-0.079645,-0.009342,1.000000


## Entrenamiento del modelo 

### Modelo bosque aleatorio random forest regressor


In [140]:
# se transforman las caracteristicas categoricas (con ordinal encoder)

In [141]:
# Se utiiza  OrdinalEncoder para transformar variables categoricas

encoder =  OrdinalEncoder()
df_ordinal = pd.DataFrame(encoder.fit_transform(df), columns = df.columns)
df_ordinal

,price,vehicletype,registrationyear,gearbox,power,model,mileage,fueltype,brand,notrepaired
0,165.0,7.0,66.0,1.0,0.0,115.0,12.0,6.0,37.0,0.0
1,156.0,5.0,7.0,1.0,0.0,127.0,0.0,6.0,10.0,0.0
2,95.0,4.0,67.0,1.0,0.0,115.0,12.0,6.0,37.0,0.0
3,360.0,7.0,89.0,1.0,0.0,132.0,12.0,2.0,27.0,0.0
4,0.0,5.0,63.0,1.0,0.0,83.0,12.0,6.0,24.0,0.0
...,...,...,...,...,...,...,...,...,...,...
33362,806.0,0.0,80.0,1.0,0.0,47.0,11.0,6.0,20.0,0.0
33363,142.0,5.0,70.0,0.0,0.0,83.0,10.0,6.0,24.0,1.0
33364,479.0,4.0,78.0,0.0,0.0,59.0,12.0,6.0,20.0,0.0
33365,600.0,7.0,78.0,1.0,3.0,11.0,12.0,2.0,2.0,0.0


In [142]:
# Se segmentan los datos en tres conjuntos (entrenamiento (60%), validacion (20%)y prueba (20%)). El precio es la variable objetivo
features_ord = df_ordinal.drop('price', axis=1)
target_ord = df_ordinal['price']
features_ord_train, features_ord_valid, target_ord_train, target_ord_valid = train_test_split(features_ord, target_ord, test_size = 0.4, random_state= 12345)
features_ord_test, features_ord_valid, target_ord_test, target_ord_valid  = train_test_split(features_ord_valid, target_ord_valid, test_size = 0.5, random_state= 12345)


In [143]:
#Entrenamiento y seleccion del mejor modelo de bosque aleatorio

best_score_random_forest = 10000000000
best_est_random_forest = 0
best_depth_random_forest = 0
for est in range(1, 12): # selecciona el rango del hiperparámetro
    for depth in range(1,12):
        model_forest = RandomForestRegressor(random_state=54321, max_depth= depth, n_estimators=est) # configura el número de árboles
        model_forest.fit(features_ord_train, target_ord_train) # entrena el modelo en el conjunto de entrenamiento
        predictions_forest = model_forest.predict(features_ord_valid) 
        
        rsme = mean_squared_error(target_ord_valid, predictions_forest)# calcula la puntuación de rsme en el conjunto de validación
        if rsme < best_score_random_forest:
            best_model_forest = model_forest
            best_score_random_forest =  rsme# guarda la mejor puntuación RSME en el conjunto de validación
            best_est_random_forest = est# guarda el número de estimadores que corresponden a la mejor puntuación de RSME
            best_depth_random_forest = depth # guarda la la profundidad que corresponde a la mejor puntacion de RSME

print("Mejor modelo 'Bosque Aleatorio':")
print("RSME:", best_score_random_forest, "", "Profundidad:",  best_depth_random_forest, "n_estimators:", best_est_random_forest)






Mejor modelo 'Bosque Aleatorio':
RSME: 32914.25379719474  Profundidad: 11 n_estimators: 11


In [144]:

#best model

import time
inicio = time.time()
    
model_forest = RandomForestRegressor(random_state=54321, max_depth= best_depth_random_forest, n_estimators=best_est_random_forest)
    
model_forest.fit(features_ord_train, target_ord_train)
predictions_forest = model_forest.predict(features_ord_valid) 
rsme = mean_squared_error(target_ord_valid, predictions_forest)


# Registrar el tiempo actual después del bloque de código
fin = time.time()
# Calcular la diferencia de tiempo
tiempo_total = fin - inicio
print("Valores del mejor modelo de bosque aleatorio")
print("tiempo de ejecucion:" , tiempo_total)
print("RSME:", rsme)

Valores del mejor modelo de bosque aleatorio
tiempo de ejecucion: 0.349564790725708
RSME: 32914.25379719474


### Modelo LIGHT gvm

In [146]:
#Entrenamiento del modelo LGVM

best_rsme_lgb = 10000000000
best_est_lgb = 0
best_num_leaves_lgb = 0
best_learning_rate_lgb = 0

for est in range(2, 5): # selecciona el rango del hiperparámetro
    for leaves in range(32, 35):
        for rate in np.linspace(0.1,0.5,3):
            clf = lgb.LGBMClassifier(learning_rate=rate, num_leaves = leaves, n_estimators = est)
            clf.fit(features_ord_train, target_ord_train)
            predictions_lgb =clf.predict(features_ord_valid)
            rsme_lgb = mean_squared_error(target_ord_valid, predictions_lgb)

            if rsme_lgb < best_rsme_lgb:
                best_clft = clf
                best_rsme_lgb  =  rsme_lgb # guarda la mejor puntuación RSME en el conjunto de validación
                best_est_lgb = est# guarda el número de estimadores que corresponden a la mejor puntuación de RSME
                best_num_leaves_lgb = leaves # guarda la la profundidad que corresponde a la mejor puntacion de RSME
                best_learning_rate_lgb = rate


print("Mejor modelo 'LGVM':")
print("RSME:", best_rsme_lgb, "", "Leaves:",  best_num_leaves_lgb, "n_estimators:", best_est_lgb, "", 'Learning rate', best_learning_rate_lgb)




Mejor modelo 'LGVM':
RSME: 78737.86469883128  Leaves: 32 n_estimators: 3  Learning rate 0.30000000000000004


In [147]:
## Modelo Lgvm con los mejores parametros

inicio_lgvm = time.time()
    
clf = lgb.LGBMClassifier(learning_rate=best_learning_rate_lgb, num_leaves = best_num_leaves_lgb, n_estimators = best_est_lgb)
clf.fit(features_ord_train, target_ord_train)
predictions_lgb =clf.predict(features_ord_valid)
rsme_lgb = mean_squared_error(target_ord_valid, predictions_lgb)
            

# Registrar el tiempo actual después del bloque de código
fin_lgvm = time.time()
# Calcular la diferencia de tiempo
tiempo_total_lgvm = fin_lgvm - inicio_lgvm
print("Valores del mejor modelo Lgvm")
print("tiempo de ejecucion:" , tiempo_total_lgvm)
print("RSME:", rsme_lgb)   
    
    
            

Valores del mejor modelo Lgvm
tiempo de ejecucion: 26.642438650131226
RSME: 78737.86469883128



LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [148]:
  del features_ord_train

### Modelo regresion lineal

In [149]:
# se transforman las caracteristicas categoricas (con OHE)

In [150]:
# Se aplica OHE a las columnas categoricas

df_ohe = pd.get_dummies(df, drop_first = False, columns = ['vehicletype', 'gearbox', 'model', 'fueltype', 'brand', 'notrepaired'])
df_ohe.head()


,price,registrationyear,power,mileage,vehicletype_bus,vehicletype_convertible,vehicletype_coupe,vehicletype_other,vehicletype_sedan,vehicletype_small,...,brand_skoda,brand_smart,brand_subaru,brand_suzuki,brand_toyota,brand_trabant,brand_volkswagen,brand_volvo,notrepaired_no,notrepaired_yes
0,480,1993,0,150000,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
14,450,1910,0,5000,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
31,245,1994,0,150000,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,1,0
36,1500,2016,0,150000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
39,0,1990,0,150000,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0


In [151]:
# Se separan las caracteristicas del objetivo. 

target_ohe = df_ohe['price']
features_ohe = df_ohe.drop(['price'], axis=1)

In [152]:
# Se segmentan los datos en tres conjuntos (entrenamiento (60%), validacion (20%)y prueba (20%)). 

features_ohe_train, features_ohe_valid, target_ohe_train, target_ohe_valid = train_test_split(features_ohe, target_ohe, test_size = 0.4, random_state= 12345)
features_ohe_test, features_ohe_valid, target_ohe_test, target_ohe_valid  = train_test_split(features_ohe_valid, target_ohe_valid, test_size = 0.5, random_state= 12345)


In [154]:
# Se escalan las caracteristicas numericas

# Se escalan las caracteristicas numericas (por ser un modelo basado en regresion)

numeric = ['registrationyear', 'power','mileage']
scaler = StandardScaler()
scaler.fit(features_ohe_train[numeric])
features_ohe_train[numeric] = scaler.transform(features_ohe_train[numeric])
features_ohe_valid[numeric] = scaler.transform(features_ohe_valid[numeric])
features_ohe_test[numeric] = scaler.transform(features_ohe_test[numeric])

/tmp/ipykernel_26/1838371082.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_ohe_train[numeric] = scaler.transform(features_ohe_train[numeric])
/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)


In [155]:
# Se entrena el modelo y se calculan (RECM y velocidad)

inicio_lr = time.time()

model_linear = LinearRegression()# inicializacion del constructor de modelo
model_linear.fit(features_ohe_train, target_ohe_train) # entrenamiento del modelo en el conjunto de entrenamiento
predictions_linear = model_linear.predict(features_ohe_valid) #  predicciones del modelo en el conjunto de validación
rsme_lr=mean_squared_error(target_ohe_valid, predictions_linear)

fin_lr = time.time()
tiempo_total_lr = fin_lr - inicio_lr

print("tiempo de ejecucion Modelo de regresion:" , tiempo_total_lr)
print("RSME:", rsme_lr) 


tiempo de ejecucion Modelo de regresion: 1.5386395454406738
RSME: 9.401039361129412e+24


## Análisis del modelo

- Del analisis de RSME y tiempos de ejecucion requeridos, se deduce que el mejor modelo es el de bisque aleatorio ya que cuenta con el menor valor de RSME y de tiempo de ejecución con los mejores parametros encontrados

# Lista de control

Escribe 'x' para verificar. Luego presiona Shift+Enter

- [x]  Jupyter Notebook está abierto
- [ ]  El código no tiene errores- [ ]  Las celdas con el código han sido colocadas en orden de ejecución- [ ]  Los datos han sido descargados y preparados- [ ]  Los modelos han sido entrenados
- [ ]  Se realizó el análisis de velocidad y calidad de los modelos